# Overview

In order to be able to submit python code to generate spark workloads we need to setup the following prerequisites:
1. Install Java
2. Install Apache Spark
3. Install Programming Language
4. Install Language Bindings

If we will be leveraging the spark/kubernetes integration built into the SparkContext we will need to setup:

5. Install Kubectl
6. Configure Kubernetes To Host Apache Spark

# 1. Install Java

According to the documentation Apark 3.1.1 requires Java 8/11. In the case of the openjdk, we will see a version of 1.8.x coresponding to Oracle version 8.

In [1]:
# Check the java version
! java -version

java version "1.8.0_291"
Java(TM) SE Runtime Environment (build 1.8.0_291-b10)
Java HotSpot(TM) 64-Bit Server VM (build 25.291-b10, mixed mode)


# 2. Install Apache spark
Apache Spark is supplied as an archive file as opposed to an installation program (like an .exe, .msi, .rpm, etc). The archive needs to be downloaded and extracted to a directory location with no spaces.

In my case, the archive has been extracted to the following directory:
```
c:\spark\spark-3.1.1-bin-hadoop2.7
```

# 3. Install Programming Language
Accodring to the documentation Spark has the following compatabilities for it's landuage bindings:
- Scala 2.12
- Python 3.6+
- R 3.5+

Insure a compatable version is installed. In our case we are using python.

In [2]:
import sys
print (sys.version)

3.6.8 (tags/v3.6.8:3c6b436a57, Dec 24 2018, 00:16:47) [MSC v.1916 64 bit (AMD64)]


# 4. Install Language Bindings
There are a few python libraries we will be using today:
- **findspark** - a utility which adds spark to the PATH variable. By doing so, it allows the pyspark library to find and use the spark libraries and binaries.
- **pyspark** - the python spark library which gives us access to spark through python.
- **py4j** - a library which enables Python programs running in a Python interpreter to dynamically access Java objects in a Java Virtual Machine. This library is consumed by pyspark.

We can check the installed version of these libraries with the following commands:

In [3]:
# Check if pyspark is intalled

! pip list | findstr "findspark"

findspark           1.4.2


In [4]:
# Check if pyspark is intalled

! pip list | findstr "pyspark"

pyspark             3.1.1


In [5]:
# Check if py4j is intalled

! pip list | findstr "py4j"

py4j                0.10.9


# 5. Install And Configure Kubectl
Kubectl is a command line utility used for interacting with a kubernetes cluster. This utility is used by the spark libraries to submit work to the kubernetes cluster.

## 5.1. Install Software

There are a number of ways to install kubectl. For windows useres, the easiest and fully featured way is to use the [chocolatey](https://kubernetes.io/docs/tasks/tools/install-kubectl-windows/#install-on-windows-using-chocolatey-or-scoop) package installation manager. For linux users, you must use the distro specific package manager (YUM, apt, etc.)

Once the installation is complete we can run the following command to check the version of our kubectl command.

In [3]:
! kubectl version --client

Client Version: version.Info{Major:"1", Minor:"5", GitVersion:"v1.5.2", GitCommit:"269f928217957e7126dc87e6adfa82242bfe5b1e", GitTreeState:"clean", BuildDate:"2017-07-03T15:31:10Z", GoVersion:"go1.7.4", Compiler:"gc", Platform:"linux/amd64"}


## 5.2. Configure Client

Once installed, we need to configure kubectl so that it can connect to the kubernetes cluster.

This is done by creating and editing the "kubeconfig" file. This is a file located in your user's "home directory".

We first create the .kube directory in our user directory. For example, on windows we create the following:

In [ ]:
! cd %USERPROFILE% & mkdir .kube 2> NUL

We then create the kubeconfig file. For simple POC installations we can copy it from the master node of our kubernetes cluster. Setting up kubernetes is outside our scope.

In my case, the file resembled the following:

```
[root@os004k8-master001 ~]# cat ~/.kube/config
apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUM1ekNDQWMrZ0F3SUJBZ0lCQURBTkJna3Foa2lHOXcwQkFRc0ZBREFWTVJNd0VRWURWUVFERXdwcmRXSmwKY201bGRHVnpNQjRYRFRJeE1EUXlOekF4TkRBd05Wb1hEVE14TURReU5UQXhOREF3TlZvd0ZURVRNQkVHQTFVRQpBeE1LYTNWaVpYSnVaWFJsY3pDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBSzdPCmlYc1NnYkZkQ21HdFA5WEdDT0FpNTlkUW5oTWU0TnplVzgvYUVwQVc5YTVEbHZ2aEx2cWQxeXpQZ29GYy9yWmMKbGorditQc3ZZTXM1UjFjRjBvT2o5Tk95VkVGVHVWVTZMTEVsc3FxWkthaHd3QW1Jbk9RaERKOStIMjE2U3BVOQpjTzVuMWxlUncwUGkreGJRNnNKeG5GcVRyUjYrRUlVODhmcHJFaG1SMW1TUFcwSm1ranFOcGhUNkQvUWxEdUptCkdWOWdEeFdwbEVBb2IvNW5RYkM1ay9ra0xpOGVEUFErWDVOWGZTeXVVY0NtVzcra1RDbEdUMFJSRzlDcFpYVHAKSHM2TGRidlZmUGU4Nm00TmQ5cGcrem1OTld0STBsZ3AvZVBodmJqMTNreTg2SmJ2VkVodmhVa3FGL2c2TmVrKwpGNVdjWDc3REt1UnhwOHpJRXNNQ0F3RUFBYU5DTUVBd0RnWURWUjBQQVFIL0JBUURBZ0trTUE4R0ExVWRFd0VCCi93UUZNQU1CQWY4d0hRWURWUjBPQkJZRUZGUXJZOVdEcm9kRXFSWUk3L0c0aGY2UEZrUi9NQTBHQ1NxR1NJYjMKRFFFQkN3VUFBNElCQVFDWDhCaTFxNGtnOTBJaE1QTkxJVm50MUhEZUI2Q3Arc2cyMFFQN0VSMTlkQ09RNWQ2UgpubEFVcnNWWFJTbXNpckluT2RaNHBabWl6OWw4QnV3dVRTdEwveTVlOTRpRlkyeVZVSUJpOWVsQTJ3em0zRHNTCkhzRDB3NTNaWlppcENoZjN0Yml6ZlpnWmFqZjBaREd4aUFVVzBHNDN5OGRNVDJTWkQrM2ltYktUd0tMUFF6OEoKUDEvdGlxKzB1cC9GY1BFTmYreCtxeE5xaytZeDJaOHIydHFlU3BENyt0bGFYVXZpdDZSVWFzMkVTWjZwVEo0SgpqRmN1M0lGMHg1dU5zNmNLUXkxbnlGMnlwTWlFeExrNXJWWGFnK1p5enZSemN5SEpIaDdOcjdJVnR5UDFJWklJCllkSDR2RjZaaXQxbzBodnhjbGdSTDV6QkU4NzZRclVlVjVmWAotLS0tLUVORCBDRVJUSUZJQ0FURS0tLS0tCg==
    server: https://15.4.7.11:6443
  name: kubernetes
contexts:
- context:
    cluster: kubernetes
    user: kubernetes-admin
  name: kubernetes-admin@kubernetes
current-context: kubernetes-admin@kubernetes
kind: Config
preferences: {}
users:
- name: kubernetes-admin
  user:
    client-certificate-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURJVENDQWdtZ0F3SUJBZ0lJRk5ZVUR5N0xwWXd3RFFZSktvWklodmNOQVFFTEJRQXdGVEVUTUJFR0ExVUUKQXhNS2EzVmlaWEp1WlhSbGN6QWVGdzB5TVRBME1qY3dNVFF3TURWYUZ3MHlNakEwTWpjd01UUXdNVEJhTURReApGekFWQmdOVkJBb1REbk41YzNSbGJUcHRZWE4wWlhKek1Sa3dGd1lEVlFRREV4QnJkV0psY201bGRHVnpMV0ZrCmJXbHVNSUlCSWpBTkJna3Foa2lHOXcwQkFRRUZBQU9DQVE4QU1JSUJDZ0tDQVFFQTRreWhLTEJVSndXMEVEdmEKeERLemQrcGlicTJreEVJN01sNmE5THJOSXN1OGJXaGgvNlp3MWdGUFJ1T0N4bVYyaTYvUC9lM05rby9hdWhHaAp0WVBWRFhqaUVPbkR5VUl2amhlaFNIVFo4bi9TSXVTdkpnK2t1UVBNWG0waElyT1F0MEtUR3F2cWwwQXNxTVQ1Ck1PRkNMSWdDR0ZCN2VkdHMrd2JmdGhSeHpVaVR4QVBZVWUwU0lQMGRlS2VzTkpqcDBxUG5BVVVpWFJTUjRUZEwKdnRSZnJCdjR4dXpFdVhYN2N1LzZMZVRDbXRQZzE5L1BtWVFoRHF2KzljZmZNenJGN3p3aWhDTUMzSEtZaWszdApmZmxYUkcyNkxYZFpyZXZES0NuSXhleGQ5cWhiVkxsUGl4NVRBY28yM0dGSXY4N1Y3QjBRb1ZXb3JGRVFuYTZmCnJxNWtkd0lEQVFBQm8xWXdWREFPQmdOVkhROEJBZjhFQkFNQ0JhQXdFd1lEVlIwbEJBd3dDZ1lJS3dZQkJRVUgKQXdJd0RBWURWUjBUQVFIL0JBSXdBREFmQmdOVkhTTUVHREFXZ0JSVUsyUFZnNjZIUktrV0NPL3h1SVgranhaRQpmekFOQmdrcWhraUc5dzBCQVFzRkFBT0NBUUVBbDB1bDRjVmREbGorVm1ZMElWRVBaRVlDVjdydTF4K3pIS2dyClJNM2NQZFZUa244UXZtT1B5WXoxZFJZSWU2cTFXRDNmZENQb0JxemorVmxvU0VJZGRIWWVhNGpHQSs3VUhOaG4KNW5TWk9oUjRMc2RITUFLd25ZdWdZK2xadm5HdktFQWJVUkN6cS9TWlM2bkhqRTB2d3I3YVhqMU5JZDlMazQwawphZDdWaGJPK0Y0NW1MRGhHdTQ2TmtlaDhQSHFmUndSNWd4K3pKRlFVT1hpZ0NVSk9HRWJjYklxMDBGaUV3RWxMCnVpd01EY2RZY3hYbmUvQzlQNktZR1ZaSXpJS1Q5aWIyWFBoZG1PcmxlWXNUcGlKSTBmajFuRlRMOXNFWk83Q2kKdjF0VmRSOW43ZjR3VHk0NGhFdzBvVnRwMlRQc1ZaT0dkMWs1OVN0aDlNcHd4cUhYUEE9PQotLS0tLUVORCBDRVJUSUZJQ0FURS0tLS0tCg==
    client-key-data: LS0tLS1CRUdJTiBSU0EgUFJJVkFURSBLRVktLS0tLQpNSUlFcEFJQkFBS0NBUUVBNGt5aEtMQlVKd1cwRUR2YXhES3pkK3BpYnEya3hFSTdNbDZhOUxyTklzdThiV2hoCi82WncxZ0ZQUnVPQ3htVjJpNi9QL2UzTmtvL2F1aEdodFlQVkRYamlFT25EeVVJdmpoZWhTSFRaOG4vU0l1U3YKSmcra3VRUE1YbTBoSXJPUXQwS1RHcXZxbDBBc3FNVDVNT0ZDTElnQ0dGQjdlZHRzK3diZnRoUnh6VWlUeEFQWQpVZTBTSVAwZGVLZXNOSmpwMHFQbkFVVWlYUlNSNFRkTHZ0UmZyQnY0eHV6RXVYWDdjdS82TGVUQ210UGcxOS9QCm1ZUWhEcXYrOWNmZk16ckY3endpaENNQzNIS1lpazN0ZmZsWFJHMjZMWGRacmV2REtDbkl4ZXhkOXFoYlZMbFAKaXg1VEFjbzIzR0ZJdjg3VjdCMFFvVldvckZFUW5hNmZycTVrZHdJREFRQUJBb0lCQVFDRm1aM3ZhcUpUNDE4WgowbXhtTmxTVDhCRXFyR05BNXFVU0tKNHM2M2VlR2svays2SVhqd2pBbjBkUlg1SGVWUnAyZjFZNDRNTGp5di9qCnAvTWpPWEIzb3EwOE5QWThoRkdVL3NoZUFYYUdYd1lOU1p0SnlpU0NnZ2FPbkxqYVJzNmlFckRNSjBDNUNNOFUKMkMwUnVYc2ExVFQvYTh4cDlheFNTdzJaN2xuOVNjWWpIUGhNRVhCT0FuOEJ1TDZpdUNlWVBkVm11dWtmREI0ZQpGSFBJYlBTMXltNVBycEl2Q1M3WnQ3YkRMUTNNczNMazcyNklkbll3U3c2akxOVitFaDhsYitZM3VrREZIWmlqCllZdmgvT0w4TmF2RDBUcUVhVUNSSVlHTWpDTi9ZUFRMZkcxMDQzY25wRWEydGNrOTBSdzNJUGsrZm9pRWlRcDAKWU55UW1zNVpBb0dCQVA2K1VXako0bWNkUlpZK29NRERWK09yM1NyZ0FOWHd6Um8xaDU2WmptR0tFTWU0WGVFKwoveHdabzB1M1ZibnhESi9qMWVDWTJGaFVDWjRRVGtzMTdVaWpkeFpHNlNQODdYTVpDbnB0bXNGRE5VZFlMRGIrCnZ3OWt5MWh6MGlpUEMrVlNCMS9Qbzkyc3NVWGNWOFJ2d1hlTDBxZzFSaDhhMXd4VEg4T0VBb0cxQW9HQkFPTnEKWkttTkxZY3RLN2JkUFE5ei9PNW9wQmZXZjN2OVRIRW03YkdtK3BpMERMWjNFdFlUYmV2SjNvODZLOERHdVZlRApXY0NVN3lZMWZRSkNPT1dxRytxLzNPd3FLdGRYVVFYY3BnSHE5ZzdxSGl5c3g5SmEvTXJGM09PbzNmQTV2RjFlCkh1azROcTlhSGRRamtqd0VQTnlqUVZqdnRGNUJpNWpROVJzRzkxajdBb0dCQU9CVTF6b3hkdWdaVmZ0dVlYLy8KUm5tZkVxQThMV0E1czFUZVNQVDVBdktRZENCOUVtZFdZNi9xUCt4Z29lU2lDdDBPOVh6WS9NUWtGV2dWc2V6MwpxMkxwV09qTVAybEt4Z3d5eEJCMlR1N2NjYUhIV3dCL0s5K3hnb1Q5VWRnWHlJRUUwS3VtSGdjTG1mME1nbG5oCmxIempLL3FuRjM0eTA2ZnQxNjRFcFhSMUFvR0FWbTU5UzZPSWhqTzR4WXRZUmFORG9pakxGUFJGQjQ3Qm1zKzgKenlRellnNHNOcjBTdjZJcXlNNjBjNk1jTmV4UlROMXpJeXAzcWdIeVBRV0dNbU53c011eTMxODNZUmlpaHFQUgp4RERVeTk1WEVZdXBkbDEyYm1xOUJSbU5JRndSSnpzNmJGRWRGNUFjZmgvMjJEQ1VJUmM3SXQ0WG95c3VMeXBiCnZhWHNKRGtDZ1lCcldYUzdheVJzaXhJSEdFbWVYbXBQbWVyS2ZOL3FIRjRxdzI4Y3M4YkVEbmxyMEV6T3VONGcKblVyQ2dFbkVnalFhZUI5dGZ1NjBZTmorKzBrRWJyQUFtNTc1RGZXVXZPcWpJdzA0RlZQeVB2Vy85L2xENnpmdAo2aFQyMXNkODlQK1VFc29yb0hHWndhTE1CT2x0U3ZzRHJlZGNMang0ckRvdUdJZnF6OUdCdVE9PQotLS0tLUVORCBSU0EgUFJJVkFURSBLRVktLS0tLQo=
```

Once the kubeconfig file is created, our client is configured. To see it is properly configured, we can execute the following commands to get information about our cluster and the nodes in the cluster.

In [4]:
! kubectl cluster-info

Kubernetes master is running at https://15.4.7.11:6443
CoreDNS is running at https://15.4.7.11:6443/api/v1/proxy/namespaces/kube-system/services/kube-dns

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [5]:
! kubectl get node

NAME                           STATUS    AGE
os004k8-master001.foobar.com   Ready     220d
os004k8-worker001.foobar.com   Ready     220d
os004k8-worker002.foobar.com   Ready     220d
os004k8-worker003.foobar.com   Ready     220d


# 6. Configure Kubernetes To Host Apache Spark

In order for our kubernetes cluster to successfully run a spark cluster we need to do a few things:
1. Configure RBAC - We will need to set permissions so that our jupyter notebook and spark components have the appropriate permissions.
2. Build containers - We will need to build the contaienrs which host our spark cluster nodes.

## 6.1. Configure Kubernetes RBAC
There are many ways to configure the RBAC of the kubernetes cluster. We are taking the simplest route posisble. We will define the follwing kubernetes objects:
- Namspace named "spark" which will container our spark related infrastructure
- ServiceAccount named spark-sa to serve as an identity to invoke commands as
- ClusterRole named spark-role with required permissions
- ClusterRoleBinding named spark-role-binding which attached the permissions defined in the role with the service account

We will put our configurations into a kubernetes manifest file which was defined as follows:

```
---
kind: Namespace
apiVersion: v1
metadata:
  name: spark
---
kind: ServiceAccount
apiVersion: v1
metadata:
  name: spark-sa
---
kind: ClusterRole
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  namespace: default
  name: spark-role
rules:
  - apiGroups: [""]
    resources: ["pods", "services", "configmaps" ]
    verbs: ["create", "get", "watch", "list", "post", "delete"  ]
---
kind: ClusterRoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: spark-role-binding
subjects:
  - kind: ServiceAccount
    name: spark-sa
    namespace: spark
roleRef:
  kind: ClusterRole
  name: spark-role
  apiGroup: rbac.authorization.k8s.io

```

We can apply the configuration using our kubectl command

```
! kubectl apply -f spark_rbac.manifest
```

## 6.2. Build Spark Containers For Kubernetes

In order to run Apache Spark on kubernetes, Docker container images need to be prepared to serve as the master/slave notes of the Spark cluster. Inside this container image we will install our spark binaries etc. As we will see, when we create our SparkContex we specify this container when it is created.

Spark (starting with version 2.3) ships with a Dockerfile that can be used for this purpose. These containers are debian based and the Dockerfile will install the latest version of python. This version of python neets to match the major version being run on the client which is hosting the jupyter noatbook. In other words, if we are running python 3.6 in our jupyter notebook, we need to ensure our docker container has the same version installed. 

In most cases one will need to modify or extend this image. Not only to modify the version of python being installed but also to modify the python packages which are installed on the spark nodes. Any packages we expect to execute on a cluster node will need to be installed in the docker image. This includes any software we might be using to store data, train machine learning algorithms, or perform optimizations.

I have prepared a container which runs python 3.6 and is packaged with the necessary packages to execute these notebooks.

For more information see the [Kubernetes 3.1.1 documentation](https://spark.apache.org/docs/3.1.1/running-on-kubernetes.html#docker-images).